# ISIBrno-AIMT
ISIBrno-AIMT is the winner of the [Will Two Do?](https://physionet.org/content/challenge-2021/1.0.3/sources/) challenge from [physionet.org](https://physionet.org/) 

[Source Code](https://physionet.org/static/published-projects/challenge-2021/1.0.3/sources/ISIBrnoAIMT.zip)
[Paper](https://www.cinc.org/archives/2021/pdf/CinC2021-014.pdf)
[leaderboard](https://moody-challenge.physionet.org/2021/results/)

In [1]:
import pickle
import os
import sys
import torch
import pandas as pd
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, jaccard_score, confusion_matrix, precision_score, recall_score, accuracy_score

from models.m02_ISIBrnoAIMT_BagOfWords.train import *
from models.m02_ISIBrnoAIMT_BagOfWords.model import NN
from models.m02_ISIBrnoAIMT_BagOfWords.dataset import PtbXlDataset

In [2]:
os.chdir('..')

### Bag of Words with 50 Words

In [3]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

n_BoW = 50
dataset = PtbXlDataset('data_ptb-xl/', 'test', n_BoW)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

model = NN(n_BoW).to(device)
model.load_state_dict(torch.load(f'models/m02_ISIBrnoAIMT_BagOfWords/model_{n_BoW}_BoW.pt'))

<All keys matched successfully>

In [4]:
y, pred = get_predictions(model, dataloader)
bow_columns = pd.read_csv('data_ptb-xl/bag_of_words/train_50_BoW.csv').columns[1:]

In [5]:
y = pd.DataFrame(y, columns=bow_columns)
pred = pd.DataFrame(pred, columns=bow_columns)

In [6]:
f1 = f1_score(y, pred, average='weighted')
iou = jaccard_score(y, pred, average='weighted')

print(f'F1: {round(f1, 3)}\nIOU: {round(iou, 3)}')

F1: 0.314
IOU: 0.215


In [57]:
performance_data = []

for label in y.columns:
    f1 = f1_score(y[label], pred[label])
    precision = precision_score(y[label], pred[label], zero_division=0)
    recall = recall_score(y[label], pred[label])
    accuracy = accuracy_score(y[label], pred[label])
    
    performance_data.append({'Label': label, 'f1': f1, 'precision': precision, 'recall': recall, 'accuracy': accuracy})

performance_summary = pd.DataFrame(performance_data)

performance_summary

,Label,f1,precision,recall,accuracy
0,46,0.954214,0.921795,0.988996,0.968239
1,abnormal,0.431974,0.275489,1.000000,0.275489
2,block,0.298747,0.175604,1.000000,0.175604
3,ecg,0.547794,0.377215,1.000000,0.377215
4,infarction,0.261652,0.150518,1.000000,0.150518
5,inferior,0.228338,0.128884,1.000000,0.128884
6,lead,0.226169,0.127503,1.000000,0.127503
7,left,0.515291,0.347066,1.000000,0.347066
8,non,0.200828,0.111623,1.000000,0.111623
9,normal,0.725993,0.569850,1.000000,0.569850


### Bag of Words with 20 Words

In [52]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

n_BoW = 20
dataset = PtbXlDataset('data_ptb-xl/', 'test', n_BoW)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

model = NN(n_BoW).to(device)
model.load_state_dict(torch.load(f'models/m02_ISIBrnoAIMT_BagOfWords/model_{n_BoW}_BoW.pt'))

<All keys matched successfully>

In [53]:
y, pred = get_predictions(model, dataloader)
bow_columns = pd.read_csv('data_ptb-xl/bag_of_words/train_20_BoW.csv').columns[1:]

In [54]:
y = pd.DataFrame(y, columns=bow_columns)
pred = pd.DataFrame(pred, columns=bow_columns)

In [55]:
f1 = f1_score(y, pred, average='weighted')
iou = jaccard_score(y, pred, average='weighted')

print(f'F1: {round(f1, 3)}\nIOU: {round(iou, 3)}')

F1: 0.662
IOU: 0.55


In [58]:
performance_data = []

for label in y.columns:
    f1 = f1_score(y[label], pred[label])
    precision = precision_score(y[label], pred[label], zero_division=0)
    recall = recall_score(y[label], pred[label])
    accuracy = accuracy_score(y[label], pred[label])
    
    performance_data.append({'Label': label, 'f1': f1, 'precision': precision, 'recall': recall, 'accuracy': accuracy})

performance_summary = pd.DataFrame(performance_data)

performance_summary

,Label,f1,precision,recall,accuracy
0,46,0.954214,0.921795,0.988996,0.968239
1,abnormal,0.431974,0.275489,1.000000,0.275489
2,block,0.298747,0.175604,1.000000,0.175604
3,ecg,0.547794,0.377215,1.000000,0.377215
4,infarction,0.261652,0.150518,1.000000,0.150518
5,inferior,0.228338,0.128884,1.000000,0.128884
6,lead,0.226169,0.127503,1.000000,0.127503
7,left,0.515291,0.347066,1.000000,0.347066
8,non,0.200828,0.111623,1.000000,0.111623
9,normal,0.725993,0.569850,1.000000,0.569850
